In [12]:
import cv2
import numpy as np
import utilsImage

In [3]:
class Card():
    def __init__(self, originImage):
        self.originImage = originImage
    

In [4]:
imgPath = './data/temp4.jpg'
img = cv2.imread(imgPath)
card = Card(img)

In [6]:
card

In [15]:
def getGreenColor(img):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # cv2.imshow("hsv", hsv)
    # define range of blue color in HSV
    lower = np.array([40,0,0])
    upper = np.array([85,255,255])
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower, upper)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(img,img, mask= mask)
    green_img = cv2.cvtColor(res, cv2.COLOR_HSV2BGR)
    return green_img

card.getGreenColor = getGreenColor

In [16]:
def getGreenEdges(img):
    green_img = getGreenColor(img)
    cv2.imshow('getGreenColor', green_img)
#     cv2.waitKey(0)

    #     Sharpend edge image
    green_img = cv2.cvtColor(green_img, cv2.COLOR_BGR2GRAY)
    #     green_img = cv2.GaussianBlur(green_img, (3, 3), cv2.BORDER_DEFAULT)
    green_img = sharpImg(green_img)

    #     Make edge wider
    kernel = np.ones((7, 7), np.uint8)
    #     green_sharped = cv2.GaussianBlur(green_sharped, (3, 3), cv2.BORDER_DEFAULT)
    green_img = cv2.dilate(green_img, kernel, iterations=2)

    #     Blur image
    green_img = cv2.GaussianBlur(green_img, (3, 3), cv2.BORDER_DEFAULT)
    #     cv2.imshow('GaussianBlur', green_img)
    #     cv2.waitKey(0)
    #     Get edges
    green_edges = cv2.Canny(green_img, 50, 120)
#     cv2.imshow('green_edges', green_edges)
#     cv2.waitKey(0)
    return green_edges

card.getGreenEdges = getGreenEdges

In [18]:
temp = card.getGreenColor(img)
utilsImage.showingImg(temp)

In [19]:
def getCardBorder(img):
    green_edges = getGreenEdges(img)
    contours, hierarchy = cv2.findContours(green_edges, cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    x = y = w = h = None
    for cnt in contours:
        # Get approximate polygons
        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) == 4:
            #             print("Min area: ", cv2.minAreaRect(cnt))
            x, y, w, h = cv2.boundingRect(cnt)
            print("Boundary: ", cv2.boundingRect(cnt))

            # Draw polygon
            #             drawContourBoundary(img.copy(), cnt)
            break
    return x, y, w, h
card.getCardBorder = getCardBorder